# LSTM 예제 : 감성 분석 Sentiment Analysis

- 입력된 자연어 안의 주관적 의견, 감정 등을 찾아내는 문제

- 이중 문장의 긍정/부정 등을 구분하는 경우가 많다.

- dataset: https://github.com/e9t/nsmc

    - 네이버 영화 리뷰에 대한 데이터로 평점 3점 이상이면 긍정, 미만이면 부정으로 labeling되어 있다.

In [5]:
path_to_train_file = "/content/drive/MyDrive/zero-base DS/ratings_train.txt"
path_to_test_file = "/content/drive/MyDrive/zero-base DS/ratings_test.txt"

with open(path_to_train_file, 'rb') as f:
    train_text = f.read().decode(encoding='utf-8')

with open(path_to_test_file, 'rb') as f:
    test_text = f.read().decode(encoding='utf-8')

print('Length of Train text: {} characters'.format(len(train_text)))
print('Length of  Test text: {} characters'.format(len(test_text)))

Length of Train text: 6937271 characters
Length of  Test text: 2318260 characters


In [4]:
print(train_text[:300])

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의


In [6]:
train_text[:300]

'id\tdocument\tlabel\n9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0\n3819312\t흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나\t1\n10265843\t너무재밓었다그래서보는것을추천한다\t0\n9045019\t교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정\t0\n6483659\t사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다\t1\n5403919\t막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.\t0\n7797314\t원작의'

In [8]:
train_text[:300].split('\n')

['id\tdocument\tlabel',
 '9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0',
 '3819312\t흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나\t1',
 '10265843\t너무재밓었다그래서보는것을추천한다\t0',
 '9045019\t교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정\t0',
 '6483659\t사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다\t1',
 '5403919\t막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.\t0',
 '7797314\t원작의']

In [10]:
train_text.split('\n')[:3]

['id\tdocument\tlabel',
 '9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0',
 '3819312\t흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나\t1']

In [9]:
train_text.split('\n')[3]

'10265843\t너무재밓었다그래서보는것을추천한다\t0'

In [11]:
train_text.split('\n')[3].split('\t')

['10265843', '너무재밓었다그래서보는것을추천한다', '0']

In [12]:
import numpy as np

train_Y = np.array([[int(row.split('\t')[2])] for row in train_text.split('\n')[1:] if row.count('\t') > 0])
test_Y = np.array([[int(row.split('\t')[2])] for row in test_text.split('\n')[1:] if row.count('\t') > 0])

print(train_Y.shape, test_Y.shape)
print(train_Y[:5])

(150000, 1) (50000, 1)
[[0]
 [1]
 [0]
 [0]
 [1]]


- tokenization은 자연어를 처리 가능한 최소의 단위로 나누는 것 --> 이번 예제에서는 띄어쓰기로 한다.

- cleaning은 불필요한 기호를 제거하는 것을 의미한다.

In [13]:
print("^가-힣A")

^가-힣A


In [ ]:
import re

# cleaning 함수 정의
def clean_str(string):
    string = re.sub(r"[^가-히A]", " ", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)
    string = re.sub(r"", "", string)

    return string.lower()